# Multiclass classification

We've just solved a binary classification problem. What about a **`multiclass classification task`**?

### Exercise Objectives:
- Write a Neural Network designed for a multiclass classification problem
- Observe overfitting during the model's convergence

# 1. Create the data


The **`make_blobs`** function [(see documentation)](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_blobs.html) enables to draw : 
- an arbitrary number of data sample, argument `n_samples`
- an arbitrary number of features per data sample, argument `n_features`
- an arbitrary number of categories, argument `centers`
- a distance between the categories, argument `cluster_std`

There is also the `random_state` argument that allows to draw the data deterministically, in order to reproduce the same data. Two people who choose the same random_state will have the same data.

❓ **Question** ❓ Based on the documentation, generate data with : 
- 1200 samples
- 8 features per sample
- 7 categories of data
- 8 as the distance between the categories

Select a `random_state` equal to 1.

Print the shape and check that it corresponds to (1200, 8) for `X` and (1200,) for `y`

In [3]:
# YOUR CODE HERE

❓ **Question** ❓ Using `matplotlib`, scatterplot two (arbitrary) dimensions of the input data together. Each dot should be colored with the category it belongs to.

In [4]:
# YOUR CODE HERE

❓ **Question** ❓ Repeat the operation on other dimensions, to visualy that the data are not easily separable

In [5]:
# YOUR CODE HERE

For the moment, the target `y` is a list of integers,  corresponding to the category of the input data.
It looks like `[3, 2, 2, 3, 0, 5, 1, 1, 0, 5, ...]` (in this example, we have 7 categories, from 0 to 6).

However, **`for categorical problems in Tensorflow.Keras, the target/output should be encoded`** in the following way:

```
[
[0, 0, 0, 1, 0, 0, 0], 
[0, 0, 1, 0, 0, 0, 0], 
[0, 0, 1, 0, 0, 0, 0], 
[1, 0, 0, 0, 0, 0, 0], 
[0, 0, 0, 0, 0, 1, 0], 
[0, 1, 0, 0, 0, 0, 0],
[0, 1, 0, 0, 0, 0, 0],
[1, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 1],
...
]
```

where:
* the number of rows is equal to the number of observations
* the number of columns is equal to the number of categories

👉 Each column corresponds to a category. 

👉 Each row corresponds to a target, the 1 being the category the input data belongs to.

You can view a row as a vector of probabilities.

```
Example:
| Cat 0 | Cat 1 | Cat 2 | Cat 3 | Cat 4 | Cat 5 | Cat 6 |
|-------|-------|-------|-------|-------|-------|-------|
| 0     | 0     | 0     | 1     | 0     | 0     | 0     |

means that for this given row, there is a 100% chance that the row belongs to the Cat 3
```

💡 To transform `y` to categories, use **`to_categorical`** function from Keras.

*[Remark]* In a sense, *to_categorical* works a bit like the OneHotEncoder in Sklearn but instead of encoding a categorical feature, we are now encoding the target.


❓ **Question** ❓ First print `y`, then apply the *`to_categorical`* to *`y`* and store the *`categorized version of y`* into **`y_cat`**.

In [6]:
# YOUR CODE HERE

❓ **Question** ❓ Split the dataset $X$ and $y_{cat}$  into a train and test set (size: 70/30%)

Remark : Please call the variables `X_train`, `y_train`, `X_test` and `y_test`

In [20]:
# YOUR CODE HERE

In deep learning, the data should always be standard-scaled, so as to lay _approximately_ in [-1, 1]. (We will see later why).

❓ **Question** ❓ Fit a sklearn [`StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) on the train set and transform both your train set and test set.

In [21]:
# YOUR CODE HERE

❓ **Question** ❓ Complete the following function to initialize a model that has: 
- a first layer with 50 neurons, the `relu` activation and an appropriate input dimension
- a output layer designed for a multiclassification task which outputs probabilities for each class

In [23]:
def initialize_model():
    ### Model architecture
    pass  # YOUR CODE HERE
    
    ### Model optimization : Optimizer, loss and metric 
    model.compile(loss='categorical_crossentropy', 
                  optimizer='adam', 
                  metrics=['accuracy'])
    
    return model 

### Note here that the loss is different! This is because the task is not with two categories only, therefore
### the solver is somehow different (will see it tomorrow)

model = initialize_model()

❓ **Question** ❓ How many parameters (a.k.a. weights) are there in the model?

How many parameters would a Deep Logistic Regression have with the same data?

In [24]:
# YOUR CODE HERE

❓ **Question** ❓ Fit your model on the train data with 50 epochs and plot the history

In [25]:
# YOUR CODE HERE

❓ **Question** ❓ Evaluate your model on the test set and print the accuracy

In [0]:
# YOUR CODE HERE

❓ **Question** ❓ Is this a good score? You should compare it to some sort of benchmark value. In this case, what score would a random guess give? Store this baseline score in the `accuracy_baseline` variable.

In [28]:
# YOUR CODE HERE

In [29]:
from nbresult import ChallengeResult
result = ChallengeResult('baseline',
                         accuracy=accuracy_baseline)
result.write()
print(result.check())

❗ **Remark** ❗ Wait ... If you get a closer look at the plot of the loss, it seems that the loss was still decreasing after 50 epochs. Why stopping it so soon? Let's rerun the model (with the initialization first) with 1000 epochs and plot the history

In [30]:
# YOUR CODE HERE

❓ **Question** ❓ 
- What can you say about the new loss? 
- Evaluate once again your model on the test set and print the accuracy

In [31]:
# YOUR CODE HERE

❗ **Remark** ❗ On one hand, the loss (computed on the train set) seems smaller than with 50 epochs. On the other hand, the accuracy on the test set got worse than before... 

❓ **Question** ❓ How is this phenomenon called? 

> YOUR ANSWER HERE

❗ **Remark** ❗ The overfitting occurs at some point during the iteration of the gradient descent, once the accuracy starts getting worse on the test set...

⚠️ Therefore, we should:
* either `choose a reasonable number of epochs to avoid overfitting` 
* or create what is called an `Early Stopping` criterion (cf. `Optimizers, Loss, Fitting` lecture tomorrow)

Let's visually check when the test loss increases again in practice. 

😱 Yes, by using the test set, there is a massive data-leakage..., we should create a validation set for that in reality... again, cf. tomorrow's lecture!

❓ **Question** ❓ Run the following command and plot the history

In [32]:
model = initialize_model()

history = model.fit(X_train, y_train, 
                    validation_data=(X_test, y_test), 
                    epochs=500, 
                    batch_size=16,
                    verbose=0)
plot_history(history)

❓ **Question** ❓ Plot the values of the loss and accuracy on the train set (in blue) and on the test set (in orange). What can you comment on that?

In [33]:
def plot_loss_accuracy(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='best')
    plt.show()
    
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='best')
    plt.show()

In [34]:
# YOUR CODE HERE

❓ **Question** ❓ Reproduce similar results by defining a more complex architecture that includes : 

- a first layer with 25 neurons 
- a second layer with 15 neurons
- a third layer with 10 neurons
- a final layer that outputs probability for each class



In [0]:
def initialize_model_2():
    pass  # YOUR CODE HERE


❗ **Remark** ❗ 
- We clearly see that an overfitting can happend during the training. More in our next lecture
- The model overfits as the number of parameters is very large (compare the number of weights with a logistic regression on the same data...)

**🏁 Congratulations! Commit and push your notebook**